# Aim
### To introduce a handful of tools for sequence data
#### Including:
* [**BioPython:**](https://biopython.org/)  A bioinformatics toolbox. [**Official Tutorial**](http://biopython.org/DIST/docs/tutorial/Tutorial.html) **Installation:** ```conda install -c anaconda biopython``` or ```pip install biopython```. Includes tools for: Reading and manipulating sequences, aligning sequences

In [1]:
from Bio import Seq

snippet = Seq.Seq('tatggctgtgcaggtcgtaaatcactgcata\
attcgtgtcgctcaaggcgcac')

snippet

Seq('tatggctgtgcaggtcgtaaatcactgcataattcgtgtcgctcaaggcgcac')

In [2]:
print('Complement:\t',snippet.complement())
print('Reverse Complement:\t',snippet.reverse_complement())
print('Transcription:\t',snippet.transcribe())
print('Translation ORF 1:\t',snippet.translate())
print('Translation ORF 2:\t',snippet[1:].translate())

Complement:	 ataccgacacgtccagcatttagtgacgtattaagcacagcgagttccgcgtg
Reverse Complement:	 gtgcgccttgagcgacacgaattatgcagtgatttacgacctgcacagccata
Transcription:	 uauggcugugcaggucguaaaucacugcauaauucgugucgcucaaggcgcac
Translation ORF 1:	 YGCAGRKSLHNSCRSRR
Translation ORF 2:	 MAVQVVNHCIIRVAQGA


/home/james/anaconda3/lib/python3.7/site-packages/Bio/Seq.py:2859: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [3]:
from Bio import SeqIO

fasta = SeqIO.parse('tutorial-data/pBbe1k-RFP.fasta', format='fasta')
fasta # generator, explain

<generator object FastaIterator at 0x7fe90011a9d0>

In [4]:
fasta = list(fasta) # unpack
print(len(fasta)) # how many items
plasmid = fasta[0] # access first item
plasmid

1


SeqRecord(seq=Seq('gacgtcgacaccatcgaatggtgcaaaacctttcgcggtatggcatgatagcgc...tcc', SingleLetterAlphabet()), id='pBbE1k-RFP', name='pBbE1k-RFP', description=' pBbE1k-RFP sequence 4206 bps', dbxrefs=[])

In [5]:
print(plasmid.description)

 pBbE1k-RFP sequence 4206 bps


In [6]:
plasmid_annotated = SeqIO.parse('tutorial-data/pBbe1k-RFP.gbk', format = 'gb')
plasmid_annotated = list(plasmid_annotated)[0]
plasmid_annotated

SeqRecord(seq=Seq('GACGTCGACACCATCGAATGGTGCAAAACCTTTCGCGGTATGGCATGATAGCGC...TCC', IUPACAmbiguousDNA()), id='.', name='Exported', description='synthetic circular DNA', dbxrefs=[])

In [7]:
print(plasmid_annotated.description)
print(plasmid_annotated.id)

for i in plasmid_annotated.features:
    print(i.type, i.location)

synthetic circular DNA
.
source [0:4206](+)
promoter [6:84](+)
CDS [84:1167](+)
primer_bind [103:123](-)
promoter [1398:1428](+)
protein_bind [1435:1452](+)
CDS [1492:2170](+)
terminator [2202:2274](+)
terminator [2289:2317](+)
rep_origin [2474:3063](-)
primer_bind [2554:2574](-)
terminator [3150:3245](+)
CDS [3275:4070](-)
primer_bind [3386:3406](-)
primer_bind [3996:4016](+)


In [8]:
cds1 = plasmid_annotated[84:1167]
cds2 = plasmid_annotated[1492:2170]
cds3 = plasmid_annotated[3275:4070].reverse_complement()

In [9]:
unkownProtein1 = cds1.translate() 
unkownProtein2 = cds2.translate() 
unkownProtein3 = cds3.translate() 

In [10]:
#>sp|Q9U6Y8|RFP_DISSP Red fluorescent protein drFP583 OS=Discosoma sp. OX=86600 PE=1 SV=1
rfp_seq = '''MRSSKNVIKEFMRFKVRMEGTVNGHEFEIEGEGEGRPYEGHNTVKLKVTKGGPLPFAWDI
LSPQFQYGSKVYVKHPADIPDYKKLSFPEGFKWERVMNFEDGGVVTVTQDSSLQDGCFIY
KVKFIGVNFPSDGPVMQKKTMGWEASTERLYPRDGVLKGEIHKALKLKDGGHYLVEFKSI
YMAKKPVQLPGYYYVDSKLDITSHNEDYTIVEQYERTEGRHHLFL'''

In [11]:
from Bio import pairwise2

alignments = pairwise2.align.globalxx(unkownProtein2.seq, rfp_seq)

print(f'Number of alignments: {len(alignments)}') # returns list of alignmets
top_alignment = alignments[0] # each alignment is a tuple 

print('\n\n')
print('First sequence: ',top_alignment[0][0:30])
print('Second sequence: ',top_alignment[1][0:30])
print(f'Alignment score = {top_alignment[2]}')
print(f'Alignment start = {top_alignment[3]}')
print(f'Alignment end = {top_alignment[4]}')

Number of alignments: 336



First sequence:  MA-SSED--VIKEFMRFKVRMEGS-VNGHE
Second sequence:  M-RSS--KNVIKEFMRFKVRMEG-TVNGHE
Alignment score = 193.0
Alignment start = 0
Alignment end = 261


In [12]:
import pandas as pd

alignment_df = pd.DataFrame([list(top_alignment[0]), list(top_alignment[1])],
                           index = ['pBbe1k-RFP','RFP_DISSP'])

alignment_df

,0,1,2,3,4,5,6,7,8,9,...,251,252,253,254,255,256,257,258,259,260
pBbe1k-RFP,M,A,-,S,S,E,D,-,-,V,...,H,S,T,G,A,*,-,-,-,-
RFP_DISSP,M,-,R,S,S,-,-,K,N,V,...,H,-,-,-,-,-,H,L,F,L


In [13]:
conserved_residue_count = 0
for i in alignment_df:
    if len(alignment_df[i].unique()) <2:
        conserved_residue_count += 1
        
print(f'Conservation: {round(conserved_residue_count/alignment_df.shape[1]*100, 2)} %')

Conservation: 26.05 %
